# Projet 4 : Anticiper les besoins en consommation électrique de bâtiments
*Pierre-Eloi Ragetly*

Ce projet fait parti du parcours *DataScientist* d'OpenClassrooms.

L'objectif principal est de trouver un modèle permettant de prédire **les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l'habitation.**

Pour cela nous disposons des données de la ville de Seattle pour les années 2015 et 2016. Ces données sont à récupérer sur le site kaggle.

# Partie III : Data modeling

Ce notebook a pour but de présenter le travail effectué sur la modélisation.

Nous commencerons par séparer notre jeu de données en deux parties distinctes:
- Le **training set**, qui va permettre d'entrainer les différents modèles;
- Le **testing set**, qui permettra de déterminer la performance du modèle finale.

Pour ce faire, la méthode `train_test_split()` de la classe *sklearn.model_selection* sera utilisée en réservant 20% des données pour le jeu de test.

Puis les modèles les plus courants seront entraînés et comparés afin de conserver les plus prometteurs. Au préalable, *une recherche par quadrillage* sera effectuée pour automatiser le choix des *hyperparamètres*, et les variables les plus pertinentes seront sélectionnées par **RFE** (Recursive Feature Elimination).

Après sélection des modèles les plus performants, nous affinerons encore les hyperparamètres à l'aide d'une *recherche aléatoire* cette fois ci, et nous en profiterons pour tester la pertinence de la variable *EnergyStarScore*.

Nous analyserons enfin les erreurs des modèles afin de déterminer s'il est pertinent d'utiliser une *méthode d'ensemble*, ie. combiner plusieurs modèles pour construire un modèle plus performant.

Le modèle final obtenu, nous pourrons évaluer sa performance à l'aide du jeu de test.

In [1]:
# Import des librairies usuelles
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns

In [2]:
# Change some default parameters of matplotlib using seaborn
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'axes.titleweight': 'bold'})
sns.set(style='ticks')
current_palette = sns.color_palette('RdBu')
sns.set_palette(current_palette)

In [3]:
# import data
data = (pd.read_csv('data/data_tr.csv')
          .set_index('OSEBuildingID')
          .drop(columns='ENERGYSTARScore'))
data_star = (pd.read_csv('data/data_tr.csv')
              .set_index('OSEBuildingID')
              .dropna(subset=['ENERGYSTARScore']))

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modéliser-la-consommation-totale-d’énergie" data-toc-modified-id="Modéliser-la-consommation-totale-d’énergie-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modéliser la consommation totale d’énergie</a></span><ul class="toc-item"><li><span><a href="#Créer-un-jeu-de-test" data-toc-modified-id="Créer-un-jeu-de-test-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Créer un jeu de test</a></span></li><li><span><a href="#Comparaison-des-modèles" data-toc-modified-id="Comparaison-des-modèles-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Comparaison des modèles</a></span><ul class="toc-item"><li><span><a href="#En-conservant-les-valeurs-par-défaut-des-hyperparamères" data-toc-modified-id="En-conservant-les-valeurs-par-défaut-des-hyperparamères-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>En conservant les valeurs par défaut des hyperparamères</a></span></li><li><span><a href="#En-optimisant-les-hyperparamètres-via-des-recherches-aléatoires-et/ou-par-grille" data-toc-modified-id="En-optimisant-les-hyperparamètres-via-des-recherches-aléatoires-et/ou-par-grille-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>En optimisant les hyperparamètres via des recherches aléatoires et/ou par grille</a></span></li></ul></li><li><span><a href="#Recherche-des-variables-les-plus-significatives" data-toc-modified-id="Recherche-des-variables-les-plus-significatives-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Recherche des variables les plus significatives</a></span></li><li><span><a href="#Sélection-des-cinq-meilleurs-modèles" data-toc-modified-id="Sélection-des-cinq-meilleurs-modèles-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Sélection des cinq meilleurs modèles</a></span></li><li><span><a href="#Modèle-final" data-toc-modified-id="Modèle-final-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Modèle final</a></span></li><li><span><a href="#Vérifier-la-pertinence-de-Energy-Star-Score" data-toc-modified-id="Vérifier-la-pertinence-de-Energy-Star-Score-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Vérifier la pertinence de Energy Star Score</a></span></li></ul></li><li><span><a href="#Modéliser-la-consommation-totale-d’énergie" data-toc-modified-id="Modéliser-la-consommation-totale-d’énergie-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Modéliser la consommation totale d’énergie</a></span><ul class="toc-item"><li><span><a href="#Créer-un-jeu-de-test" data-toc-modified-id="Créer-un-jeu-de-test-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Créer un jeu de test</a></span></li><li><span><a href="#Comparaison-des-modèles" data-toc-modified-id="Comparaison-des-modèles-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Comparaison des modèles</a></span></li><li><span><a href="#Recherche-des-variables-les-plus-significatives" data-toc-modified-id="Recherche-des-variables-les-plus-significatives-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Recherche des variables les plus significatives</a></span></li><li><span><a href="#Sélection-des-cinq-meilleurs-modèles" data-toc-modified-id="Sélection-des-cinq-meilleurs-modèles-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Sélection des cinq meilleurs modèles</a></span></li><li><span><a href="#Modèle-final" data-toc-modified-id="Modèle-final-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Modèle final</a></span></li><li><span><a href="#Vérifier-la-pertinence-de-Energy-Star-Score" data-toc-modified-id="Vérifier-la-pertinence-de-Energy-Star-Score-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Vérifier la pertinence de Energy Star Score</a></span></li></ul></li></ul></div>

## Modéliser la consommation totale d’énergie

Pour pouvoir modéliser la consommation totale en énergie il faut au préalable déterminer quelle sera notra valeur cible. Sous nous avons quatre variables potentielles :
- SiteEnergyUse(kBtu)
- SiteEnergyUseWN(kBtu)
- SiteEnergyUse(kBtu)_log
- SiteEnergyUseWN(kBtu)_log

Nous avons vu lors de l'ingénierie des variable qu'il était préférable de prendre la version log pour avoir une distribution se rapprochant d'une distribution normale. On peut donc déjà écarter les deux premières.

Se pose ensuite la question de savoir s'il est préférable de garder la version normalisée ou non normalisée. Pour rappel, la version normalisée et la consommation corrigée en prenant comme référence la température des trentes dernières années. Alors que la version normalisée est la consommation moyenne sur les années 2015 et 2016. Dans le contexte de réchauffement climatique, il est fort à parier que la températures des prochaines années sera plus proche de celles de 2015 et 2016 que de la température des trentes dernières années.

Nous prendrons donc la version non normalisée **SiteEnergyUse(kBtu)_log**.

### Créer un jeu de test

In [4]:
from sklearn.model_selection import train_test_split

X = data.iloc[:, :-6].values
y = data.loc[:, 'SiteEnergyUse(kBtu)_log'].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

Le paramètre *random_state* permet de définir *le germe* (seed) du générateur de nombre aléatoires, afin qu'il génère toujours la même suite d'indices pseudo-aléatoires.

### Comparaison des modèles

Pour comparer les résultats obtenus avec les algorithmes de regression les plus courants (voir liste ci-dessous), nous utiliserons la librairie *Scikit-Learn* ainsi que la librairie *XGBoost*.
- Régression Ridge
- Régression Lasso
- Elastic Net
- Régression SVM linéaire
- Régression SVM avec noyau
- Régression kNN
- Arbre de décision
- Forêt aléatoire
- Gradient Boosting
- XGBoost
- Perceptron multi-couches

Nous utiliserons comme mesure de performance la RMSE par validation croisée.

#### En conservant les valeurs par défaut des hyperparamères

In [5]:
from functions.ml_modeling import get_models
from functions.ml_modeling import compare_models

# Create models
default_models = get_models(X_train, y_train)

# Compare models
compare_models(X_train, y_train, default_models)

,RMSE,RMSE_std,R2,R2_std
Ridge,1.271064,0.815615,-0.287782,1.675330
Lasso,1.301737,0.037429,-0.000678,0.000595
ElasticNet,1.281750,0.044597,0.030058,0.015660
LinearSVR,1.348166,0.788939,-0.385296,1.642582
SVR,0.833117,0.036242,0.588541,0.040997
KNeighborsRegressor,0.991973,0.044768,0.414600,0.077137
DecisionTreeRegressor,0.871922,0.049904,0.549237,0.052139
RandomForestRegressor,0.647185,0.011835,0.752250,0.012061
GradientBoostingRegressor,0.626153,0.019641,0.768118,0.013423
XGBRegressor,0.651344,0.020638,0.748912,0.017389


#### En optimisant les hyperparamètres via des recherches aléatoires et/ou par grille

In [6]:
# Create models
models = get_models(X_train, y_train, best_hparams=True)

# Compare models
df = compare_models(X_train, y_train, models)
df

,RMSE,RMSE_std,R2,R2_std
Ridge,0.954890,0.052254,0.460463,0.049179
Lasso,1.146331,0.135182,0.219630,0.158305
ElasticNet,0.959244,0.056865,0.455671,0.051165
LinearSVR,1.298493,0.353942,-0.054854,0.571335
SVR,0.780573,0.043703,0.637942,0.048712
KNeighborsRegressor,0.973904,0.040785,0.438066,0.051661
DecisionTreeRegressor,0.734815,0.024689,0.680798,0.016623
RandomForestRegressor,0.641131,0.013494,0.756885,0.011884
GradientBoostingRegressor,0.616372,0.014684,0.774813,0.019057
XGBRegressor,0.616543,0.020826,0.775021,0.016036


### Recherche des variables les plus significatives

Une grande qualité des forêts aléatoires est qu'elles permettent de mesurer facilement l'importance relative des variables. C'est donc tout naturellement que nous allons utiliser cet algorithme pour sélectionner les variables les plus pertinentes.

Pour cela nous allons utiliser la classe **RFECV** de *sklearn.feature_selection*. La méthode RFE (Recursive Feature Elimination) permet de supprimer les variables de manière récursives. Le modèle est d'abord entrainé sur toutes les variables, puis l'algorithme écarte à chaque itération la ou les variables (depend du pas renseigné par l'utilisateur) les moins significatives jusqu'à obtenir le nombre voulu de variables. Pour déterminer le nombre de variables à conserver, on utilise une validation croisée.

Ensuite nous ne conserverons que les variables qui sont significatives et réentrainerons les modèles avec ces variables.

In [7]:
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV

cv = KFold(n_splits=5, shuffle=True, random_state=42)
min_features = int(X_train.shape[1]/5)
forest_reg = [m for m in models if type(m).__name__=='RandomForestRegressor'][0]

rfe = RFECV(forest_reg, min_features_to_select=min_features,
            cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)
rfe.fit(X_train, y_train)
X_train_rfe = rfe.transform(X_train)
mask = rfe.support_

print('{} features have been selected'.format(rfe.n_features_))

54 features have been selected


Nous observons que les meilleurs résultats sont obtenus lorsque nous conservons toutes les variables. Nous continuerons donc avec toutes les variables.

### Sélection des cinq meilleurs modèles

In [8]:
# Get a score based on both RMSE and R2 scores
df['RMSE_minmax'] = (df['RMSE']-df['RMSE'].min())/ \
                    (df['RMSE'].max()-df['RMSE'].min())
df['R2_minmax'] = (df['R2']-df['R2'].min())/ \
                  (df['R2'].max()-df['R2'].min())
df['Score'] = df['R2_minmax'] - df['RMSE_minmax']

# Keep the 3 models with the largest score
best_models_name = df['Score'].nlargest(5).index
best_models = [m for m in models if type(m).__name__ in best_models_name]
print("The top five models are:")
for m in best_models_name:
    print(m)

The top five models are:
GradientBoostingRegressor
XGBRegressor
RandomForestRegressor
DecisionTreeRegressor
MLPRegressor


### Modèle final

Maintenant que nous avons réentrainé les modèles les plus prometteurs en ne conservant que les variables significatives, nous allons pouvoir obtenir le modèle finale.

Pour ce modèle finale, nous allons reprendre l'astuce des modèles ensemblistes, nous allons chercher à obtenir le meilleur des modèles sélectionnés en les empilant de manière à former un nouveau modèle plus performant. Pour cela, nous utiliserons la classe **StackingRegressor** de *sklearn.ensemble*.

In [9]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV

# Get a stacking ensemble of models
estimators = []
for m in best_models:
    estimators.append((type(m).__name__, m))
stack_reg = StackingRegressor(estimators=estimators,
                              final_estimator=RidgeCV(alphas=np.logspace(-2, 2, 5)))
stack_reg.fit(X_train, y_train)

# Get the results
best_models.append(stack_reg)
df = compare_models(X_train, y_train, best_models)
df

c:\users\pe.ragettli\documents\open_classrooms\data_scientist\oc_ds_project4\env\lib\site-packages\joblib\externals\loky\process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


,RMSE,RMSE_std,R2,R2_std
DecisionTreeRegressor,0.734815,0.024689,0.680798,0.016623
RandomForestRegressor,0.641131,0.013494,0.756885,0.011884
GradientBoostingRegressor,0.616372,0.014684,0.774813,0.019057
XGBRegressor,0.616543,0.020826,0.775021,0.016036
MLPRegressor,0.755391,0.046810,0.660108,0.052493
StackingRegressor,0.606206,0.016573,0.782068,0.020161


Le modèle ainsi créé est bien plus performant, mais le gain de performance est faible comparé à un temps de calcul nettement supérieur (il faut entrainer six modèles au lieu d'un seul). Nous préfèrerons donc garder les modèles basés sur le gradient boosting. Les modèles Gradient Boosting et XGBoost ont des résultats similaires, cependant, le premier étant moins complexe, il se généralisera probablement mieux.

**Nous prendrons donc le modèle Gradient Boosting comme modèle finale pour la modélisation de la consommation énergétique.**

Regardons la performance sur notre jeu de test, cette fois nous ne prendrons par la version logarithmique qui n'est utile que pour l'entrainement.

In [10]:
from sklearn.metrics import r2_score

model = best_models[-4]
y_pred = np.exp(model.predict(X_test[:, mask]))
y_true = np.exp(y_test)
r2 = r2_score(y_true, y_pred)
print('{:.2%}'.format(r2))

91.12%


### Vérifier la pertinence de Energy Star Score

Nous allons pour clore cette partie consacrée à la modélisation des consommations énergétiques, vérifier la pertinence de la variable *Energy Star Score*. Pour cela nous allons entrainer le modèle XGBoost sur deux jeux données distincts uniquement par la présence (ou non) de cette variable. Puis nous comparerons les résultats obtenus avec chaque modèles.

In [11]:
from sklearn.ensemble import GradientBoostingRegressor
from functions.ml_modeling import gboost_reg_best_params


# Split the dataset into random train and test subsets
mask_star = (data_star.columns[:-6]!='ENERGYSTARScore')
X_star = data_star.iloc[:, :-6].values
y_star = data_star.loc[:, 'SiteEnergyUse(kBtu)_log'].values
X_star_train, X_star_test, y_star_train, y_star_test = train_test_split(X_star,
                                                                        y_star,
                                                                        test_size=0.2,
                                                                        random_state=42)
# Train the models
gboost_star = gboost_reg_best_params(X_star_train,
                                     y_star_train,
                                     GradientBoostingRegressor())
gboost_no_star = gboost_reg_best_params(X_star_train[:, mask_star],
                                        y_star_train,
                                        GradientBoostingRegressor())

# Get results
r2 = []
y_pred = np.exp(gboost_star.predict(X_star_test))
y_true = np.exp(y_star_test)
r2.append(r2_score(y_true, y_pred))
y_pred = np.exp(gboost_no_star.predict(X_star_test[:, mask_star]))
r2.append(r2_score(y_true, y_pred))
df_star = pd.DataFrame({'EnergyStarScore': [True, False], 'R2': r2})
df_star

,EnergyStarScore,R2
0,True,0.732153
1,False,0.692975


On constate que l'Energy Star Score permet d'améliorer le modèle. Cependant, bon nombre de bâtiments n'ont pas ce score. Or le résultat du paragraphe 1.5 montre clairement qu'il est préférable de conserver un maximum de données, quitte à se passer de l'Energy Star Score.

## Modéliser la consommation totale d’énergie

Là encore, nous utiliserons la version log pour l'entrainement des modèles

### Créer un jeu de test

In [12]:
from sklearn.model_selection import train_test_split

X = data.iloc[:, :-6].values
y = data.loc[:, 'TotalGHGEmissions_log'].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

### Comparaison des modèles

In [13]:
# Create models
models = get_models(X_train, y_train, best_hparams=True)

# Compare models
df = compare_models(X_train, y_train, models)
df

,RMSE,RMSE_std,R2,R2_std
Ridge,0.996483,0.101850,0.556067,0.078220
Lasso,1.184473,0.259902,0.358246,0.272554
ElasticNet,1.039327,0.155522,0.514180,0.131487
LinearSVR,1.143412,0.254518,0.399282,0.260635
SVR,0.799911,0.052754,0.713396,0.043309
KNeighborsRegressor,1.071108,0.040461,0.487519,0.053448
DecisionTreeRegressor,0.834098,0.056814,0.687746,0.050743
RandomForestRegressor,0.686416,0.026358,0.789921,0.017853
GradientBoostingRegressor,0.630126,0.027362,0.822341,0.021916
XGBRegressor,0.629322,0.023766,0.823101,0.018345


### Recherche des variables les plus significatives

In [14]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
min_features = int(X_train.shape[1]/5)
forest_reg = [m for m in models if type(m).__name__=='RandomForestRegressor'][0]

rfe = RFECV(forest_reg, min_features_to_select=min_features,
            cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)
rfe.fit(X_train, y_train)
X_train_rfe = rfe.transform(X_train)
mask = rfe.support_

print('{} features have been selected'.format(rfe.n_features_))

41 features have been selected


Il semblerait que seules 41 variables soient significatives, réentrainons les modèles sur ces 41 variables.

In [15]:
# Create models
models_rfe = get_models(X_train_rfe, y_train, best_hparams=True)

# Compare models
df_rfe = compare_models(X_train_rfe, y_train, models_rfe)
df_rfe

,RMSE,RMSE_std,R2,R2_std
Ridge,0.999320,0.078970,0.554605,0.058358
Lasso,1.184473,0.259902,0.358246,0.272554
ElasticNet,1.039311,0.155490,0.514198,0.131453
LinearSVR,1.142409,0.253048,0.400674,0.258474
SVR,0.787120,0.050351,0.722612,0.040126
KNeighborsRegressor,1.061322,0.036014,0.497871,0.039544
DecisionTreeRegressor,0.838649,0.058059,0.683993,0.053463
RandomForestRegressor,0.685511,0.027146,0.790237,0.020805
GradientBoostingRegressor,0.629821,0.032831,0.822561,0.022391
XGBRegressor,0.622965,0.029126,0.826388,0.021382


Dans l'ensemble, nous obtenons de meilleurs résultats en restreignant les variables (tout réduisant les temps de calculs). Nous poursuivrons donc en ne gardant que ces 41 variables.

### Sélection des cinq meilleurs modèles

In [16]:
# Get a score based on both RMSE and R2 scores
df_rfe['RMSE_minmax'] = (df_rfe['RMSE']-df_rfe['RMSE'].min())/ \
                        (df_rfe['RMSE'].max()-df_rfe['RMSE'].min())
df_rfe['R2_minmax'] = (df_rfe['R2']-df_rfe['R2'].min())/ \
                           (df_rfe['R2'].max()-df_rfe['R2'].min())
df_rfe['Score'] = df_rfe['R2_minmax'] - df_rfe['RMSE_minmax']

# Keep the 3 models with the largest score
best_models_name = df_rfe['Score'].nlargest(5).index
best_models = [m for m in models_rfe if type(m).__name__ in best_models_name]
print("The top five models are:")
for m in best_models_name:
    print(m)

The top five models are:
XGBRegressor
GradientBoostingRegressor
RandomForestRegressor
MLPRegressor
SVR


### Modèle final

In [17]:
# Get a stacking ensemble of models
estimators = []
for m in best_models:
    estimators.append((type(m).__name__, m))
stack_reg = StackingRegressor(estimators=estimators,
                              final_estimator=RidgeCV(alphas=np.logspace(-2, 2, 5)))
stack_reg.fit(X_train_rfe, y_train)

# Get the results
best_models.append(stack_reg)
df = compare_models(X_train_rfe, y_train, best_models)
df

,RMSE,RMSE_std,R2,R2_std
SVR,0.787120,0.050351,0.722612,0.040126
RandomForestRegressor,0.685511,0.027146,0.790237,0.020805
GradientBoostingRegressor,0.629821,0.032831,0.822561,0.022391
XGBRegressor,0.622965,0.029126,0.826388,0.021382
MLPRegressor,0.731494,0.040710,0.759611,0.039721
StackingRegressor,0.616477,0.031874,0.829652,0.024336


Là encore nous conserverons le modèle Gradient Boosting.

In [18]:
model = best_models[-4]
y_pred = np.exp(model.predict(X_test[:, mask]))
y_true = np.exp(y_test)
r2 = r2_score(y_true, y_pred)
print('{:.2%}'.format(r2))

94.68%


### Vérifier la pertinence de Energy Star Score

In [19]:
# Split the dataset into random train and test subsets
mask_star = (data_star.columns[:-6]!='ENERGYSTARScore')
X_star = data_star.iloc[:, :-6].values
y_star = data_star.loc[:, 'TotalGHGEmissions_log'].values
X_star_train, X_star_test, y_star_train, y_star_test = train_test_split(X_star,
                                                                        y_star,
                                                                        test_size=0.2,
                                                                        random_state=42)
# Train the models
gboost_star = gboost_reg_best_params(X_star_train,
                                     y_star_train,
                                     GradientBoostingRegressor())
gboost_no_star = gboost_reg_best_params(X_star_train[:, mask_star],
                                        y_star_train,
                                        GradientBoostingRegressor())

# Get results
r2 = []
y_pred = np.exp(gboost_star.predict(X_star_test))
y_true = np.exp(y_star_test)
r2.append(r2_score(y_true, y_pred))
y_pred = np.exp(gboost_no_star.predict(X_star_test[:, mask_star]))
r2.append(r2_score(y_true, y_pred))
df_star = pd.DataFrame({'EnergyStarScore': [True, False], 'R2': r2})
df_star

,EnergyStarScore,R2
0,True,0.769938
1,False,0.873640


Là encore, l'Energy Star Score permet d'améliorer significativement la performance du modèle, mais le nombre de bâtiments possédant ce score est encore trop limité pour pouvoir l'utiliser.